<a href="https://colab.research.google.com/github/shemaaali/DS-Unit-4-Sprint-3-Deep-Learning/blob/main/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment/SA9_LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [34]:
import requests
import pandas as pd

In [35]:
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

In [36]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [37]:
#Shakespeare Data Parsed by Play
df_toc.head()

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...


In [38]:
import numpy as np
from lxml import html, etree
import requests
import random

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback
from tensorflow.keras.utils import get_file

In [49]:
# picks text to use in model
text = text_small
# creates orderedlist of charecters in text
chars = ' '.join(text_small)

# gets list of unique charecters
unique_chars = list(set(chars))

# creates a dictionary to map number to charecter
num_to_char = dict(enumerate(unique_chars))

# creates dictonary maping char to num
char_to_num = {value: key for key, value in num_to_char.items()}

# encodes text
text_encoded = [char_to_num[i] for i in text_small]

In [50]:
maxlen = 40
step = 3
sentences = []
next_chars = []


for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 33318


In [51]:
# looks at sentences
sentences[:5]

["That, poor contempt, or claim'd thou sle",
 "t, poor contempt, or claim'd thou slept ",
 "poor contempt, or claim'd thou slept so ",
 "r contempt, or claim'd thou slept so fai",
 "ontempt, or claim'd thou slept so faithf"]

In [52]:
# looks at next chars
next_chars[:5]

['p', 's', 'f', 't', 'u']

In [53]:
# vectorize
x = np.zeros((len(sentences), maxlen, len(unique_chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(unique_chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_num[char]] = 1
    y[i, char_to_num[next_chars[i]]] = 1

In [54]:
# build model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(unique_chars))))
model.add(Dense(len(unique_chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [55]:
# fit model 
history = model.fit(x, 
                    y,
                    batch_size=128,
                    epochs=10)

Epoch 1/10
261/261 [==============================] - 32s 121ms/step - loss: 2.5321
Epoch 2/10
261/261 [==============================] - 32s 124ms/step - loss: 2.0400
Epoch 3/10
261/261 [==============================] - 32s 121ms/step - loss: 1.8586
Epoch 4/10
261/261 [==============================] - 32s 121ms/step - loss: 1.7281
Epoch 5/10
261/261 [==============================] - 32s 121ms/step - loss: 1.6232
Epoch 6/10
261/261 [==============================] - 32s 121ms/step - loss: 1.5248
Epoch 7/10
261/261 [==============================] - 34s 130ms/step - loss: 1.4413
Epoch 8/10
261/261 [==============================] - 31s 120ms/step - loss: 1.3696
Epoch 9/10
261/261 [==============================] - 31s 120ms/step - loss: 1.3105
Epoch 10/10
261/261 [==============================] - 32s 121ms/step - loss: 1.2549


In [56]:
start_index = random.randint(0, len(text) - maxlen - 1)

In [57]:
sentence = text[start_index: start_index + maxlen]
sentence

'trength without discourse!\nFie! I should'

In [58]:
x_pred = np.zeros((1, maxlen, len(unique_chars)))
for t, i in enumerate(sentence):
    x_pred[0, t, char_to_num[i]] = 1

In [59]:
x_pred.shape

(1, 40, 62)

In [60]:
x_pred

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [61]:
sm_prediction = model.predict(x_pred)

In [62]:
# this is the probabilities for each charecter 
sm_prediction[0].sum()

0.9999999

In [63]:
best = np.amax(sm_prediction)

In [64]:
sm_prediction/best

array([[1.3034612e-04, 4.4945956e-04, 3.6819849e-02, 4.9299001e-06,
        2.4893438e-05, 1.7433306e-04, 7.1124323e-06, 2.9696910e-06,
        1.9739609e-04, 1.5459775e-06, 3.5329762e-05, 2.9551316e-05,
        2.8273876e-06, 2.9532652e-04, 8.6550490e-06, 1.0241648e-03,
        8.5324364e-06, 4.3537276e-09, 2.1995900e-03, 4.0082554e-03,
        1.0472810e-05, 1.0055753e-04, 2.6357331e-05, 4.2494180e-04,
        3.3915114e-07, 2.4837320e-04, 1.4145516e-05, 2.0070504e-05,
        2.4742671e-05, 2.1871374e-06, 1.9004417e-05, 1.1683347e-06,
        6.7441247e-04, 6.3836592e-06, 7.1808420e-11, 4.5230783e-05,
        5.3029980e-05, 5.3607544e-04, 4.9117435e-04, 2.2551890e-03,
        1.4448420e-03, 1.0431708e-02, 4.7394674e-04, 1.8406569e-04,
        1.2422989e-04, 8.8205590e-05, 2.7706370e-02, 6.5004779e-04,
        2.9575380e-03, 1.8057010e-06, 2.9856403e-05, 5.7272764e-04,
        1.9037450e-05, 7.1034003e-05, 1.0000000e+00, 9.1990252e-05,
        2.2414756e-08, 1.7962142e-05, 7.4688680e

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN